## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [22]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/1990'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [23]:
sp500.head()

,permno,start,ending,date,ret
0,20220,1976-01-15,2010-03-12,1990-01-31,-0.166667
1,19051,1979-03-01,1999-07-21,1990-01-31,-0.141243
2,61399,1984-03-01,2023-12-29,1990-01-31,-0.071864
3,47896,1976-07-01,2023-12-29,1990-01-31,-0.138075
4,52337,1979-02-08,2016-04-15,1990-01-31,-0.145161


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [24]:
mse = conn.raw_sql("""
    select 
        msenames.comnam, 
        msenames.ncusip, 
        msenames.namedt, 
        msenames.nameendt, 
        msenames.permno, 
        msenames.shrcd, 
        msenames.exchcd, 
        msenames.hsiccd, 
        msenames.ticker,
        comp.company.gsector as sector
    from crsp.msenames
    left join crsp.ccmxpf_linktable as link
        on msenames.permno = link.lpermno
        and link.linktype in ('LU', 'LC')
        and (link.linkdt <= msenames.namedt or link.linkdt is null)
        and (msenames.nameendt <= link.linkenddt or link.linkenddt is null)
    left join comp.company
        on link.gvkey = comp.company.gvkey
    """, date_cols=['namedt', 'nameendt'])

# If nameendt is missing then set to today's date
mse['nameendt'] = mse['nameendt'].fillna(pd.to_datetime('today'))

In [25]:
sector_mapping = {
    '10': "Energy",
    '15': "Materials",
    '20': "Industrials",
    '25': "Consumer Discretionary",
    '30': "Consumer Staples",
    '35': "Health Care",
    '40': "Financials",
    '45': "Information Technology",
    '50': "Communication Services",
    '55': "Utilities",
    '60': "Real Estate"
}

# Assuming 'mse' is the DataFrame with sector numbers
mse['sector_name'] = mse['sector'].map(sector_mapping)

# Fill missing 'nameendt' with today's date
mse['nameendt'] = mse['nameendt'].fillna(pd.to_datetime('today'))


In [27]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker,sector,sector_name
38409,IRVING AIR CHUTE INC,None,1962-07-02,1968-01-01,33240,10,2,2399,IHV,None,NaN
17852,CADENCE BANCORP,12739A10,2017-04-13,2021-10-28,16654,11,1,6021,CADE,40,Financials
71361,TRI BASIN RESOURCES LTD,89590210,1985-11-05,1986-07-16,78377,12,0,6790,None,None,NaN
19697,E T F SERIES SOLUTIONS,26922A48,2020-02-25,2020-06-25,17410,73,5,6726,MSUS,None,NaN
107328,PROGRESS FINANCIAL CORP,74326610,1986-07-18,1986-12-29,91257,11,3,6030,PBNK,40,Financials


In [30]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.to_csv('data/sp500_constituents.csv', index=False)

In [36]:
import pandas as pd
from wrds import Connection

# Establish a connection to WRDS
conn = Connection(wrds_username='nicklewis16')

# SQL query to retrieve unique stock data with sectors from CRSP/Compustat Merged Database
query = """
    SELECT DISTINCT
        a.permno,
        b.gvkey,
        c.gsector,
        c.gind,
        c.gsubind,
        a.ticker,
        a.comnam
    FROM 
        crsp.msenames AS a
    JOIN 
        crsp.ccmxpf_linktable AS b
        ON a.permno = b.lpermno
    AND 
        b.linktype IN ('LU', 'LC')
    AND 
        (b.linkdt <= a.namedt OR b.linkdt IS NULL)
    AND 
        (a.nameendt <= b.linkenddt OR b.linkenddt IS NULL)
    JOIN 
        comp.company AS c
        ON b.gvkey = c.gvkey
    WHERE 
        a.namedt >= '1990-01-01'
    ORDER BY 
        a.permno;
    """

# Execute the SQL query and fetch data into a DataFrame
data = conn.raw_sql(query, date_cols=['namedt', 'nameendt'])

# Close the connection
conn.close()

Loading library list...
Done


In [43]:
sector_mapping = {
    '10': "Energy",
    '15': "Materials",
    '20': "Industrials",
    '25': "Consumer Discretionary",
    '30': "Consumer Staples",
    '35': "Health Care",
    '40': "Financials",
    '45': "Information Technology",
    '50': "Communication Services",
    '55': "Utilities",
    '60': "Real Estate"
}

# Assuming 'mse' is the DataFrame with sector numbers
data['sector'] = data['gsector'].map(sector_mapping)

data.drop_duplicates(subset=['permno'], inplace=True, keep='first')
data.to_csv('data/unique_combination_crsp_tickers.csv', index=False)